In [ ]:
import pandas as pd
import yfinance as yf

In [ ]:
df = yf.download('APPL', start='2020-01-01')

In [ ]:
df

In [ ]:
def create_renko(df, brick_size):
    renko_list = []
    previous_price = df.iloc[0]['Adj Close']
    for index, row in df.iterrows():
        price = row['Adj Close']
        while abs(price - previous_price) >= brick_size:
            if price > previous_price:
                previous_price += brick_size
                renko_list.append({'Date': index, 'Close': previous_price, 'Brick': 'up'})
            else:
                previous_price -= brick_size
                renko_list.append({'Date': index, 'Close': previous_price, 'Brick': 'down'})
    renko_df = pd.DateFrame(renko_list)
    return renko_df

In [ ]:
renko_df = create_renko(df, 20)

In [ ]:
renko_df

In [ ]:
initial_balance = 10000
balance = initial_balance
position = 0

for i in range(len(renko_df)):

    # Buy signal: Upward reversal or the first upward brick
    if renko_df.iloc[i]['Brick'] == 'up' and (i == 0 or renko_df.iloc[i-1]['Brick'] == 'down'):
        if position == 0:   # Ensure we are not already holding a position
            buy_price = renko_df.iloc[i]['Close']
            position = balance / buy_price
            balance = 0
            print(f"Bought at {buy_price} on {renko_df.iloc[i]['Date']}")
    
    # Sell signal: Downward reversal
    elif renko_df.iloc[i]['Brick'] == 'down' and renko_df.iloc[i-1]['Brick'] == 'up':
        if position > 0:
            sell_price = renko_df.iloc[i]['Close']
            balance = position * sell_price
            position = 0    # Reset position
            print(f"Sold at {sell_price} on {renko_df.iloc[i]['Date']}")

if position > 0:
    balance = position * renko_df.iloc[-1]['Close']

print(f"Initial balance: {initial_balance}")
print(f"Final balance: {balance}")


In [ ]:
renko_df